In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 연예인(나무위키 사이트까지) 지정

### div 지정해서 텍스트 가져오기

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

# 크롬 드라이버 설정
options = Options()
options.add_argument("--headless=new") # 돌아가는 거거 안보이게 하는 옵션
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--lang=ko_KR")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# 연예인 목록 CSV 불러오기
artist_df = pd.read_csv("JYP_연예인목록.csv")

# 결과 저장 CSV
with open("JYP논란_크롤링.csv", "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '그룹', '연예인 이름', '텍스트'])

    for _, artist in artist_df.iterrows():
        url = str(artist['url']).strip()
        
        if not url.startswith("http"):
            # URL이 없으면 텍스트 없이 기본 정보만 기록
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])
            print(f"ℹ️ URL 없음 → 텍스트 없이 기록: {artist['연예인 이름']}")
            continue

        try:
            driver.get(url)
            time.sleep(3)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # BEFORE: 페이지 전체 텍스트
            # text = soup.get_text(separator="\n", strip=True)

            # AFTER: 특정 div 텍스트만
            target_div = soup.find("div", class_="EyS5B+jf upriic4n")
            text = target_div.get_text(separator="\n", strip=True) if target_div else ""

            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                text
            ])
        except Exception as e:
            print(f"❌ 크롤링 오류: {artist['연예인 이름']} → {e}")
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])

driver.quit()

ℹ️ URL 없음 → 텍스트 없이 기록: 시우민
ℹ️ URL 없음 → 텍스트 없이 기록: 수호
ℹ️ URL 없음 → 텍스트 없이 기록: 세훈
ℹ️ URL 없음 → 텍스트 없이 기록: 최강창민
ℹ️ URL 없음 → 텍스트 없이 기록: 예성
ℹ️ URL 없음 → 텍스트 없이 기록: 동해
ℹ️ URL 없음 → 텍스트 없이 기록: 려욱
ℹ️ URL 없음 → 텍스트 없이 기록: 규현
ℹ️ URL 없음 → 텍스트 없이 기록: 종현
ℹ️ URL 없음 → 텍스트 없이 기록: 키
ℹ️ URL 없음 → 텍스트 없이 기록: 민호
ℹ️ URL 없음 → 텍스트 없이 기록: 태민
ℹ️ URL 없음 → 텍스트 없이 기록: 유타
ℹ️ URL 없음 → 텍스트 없이 기록: 쿤
ℹ️ URL 없음 → 텍스트 없이 기록: 도영
ℹ️ URL 없음 → 텍스트 없이 기록: 텐
ℹ️ URL 없음 → 텍스트 없이 기록: 마크
ℹ️ URL 없음 → 텍스트 없이 기록: 샤오쥔
ℹ️ URL 없음 → 텍스트 없이 기록: 헨드리
ℹ️ URL 없음 → 텍스트 없이 기록: 제노
ℹ️ URL 없음 → 텍스트 없이 기록: 재민
ℹ️ URL 없음 → 텍스트 없이 기록: 양양
ℹ️ URL 없음 → 텍스트 없이 기록: 천러
ℹ️ URL 없음 → 텍스트 없이 기록: 지성
ℹ️ URL 없음 → 텍스트 없이 기록: 쇼타로
ℹ️ URL 없음 → 텍스트 없이 기록: 은석
ℹ️ URL 없음 → 텍스트 없이 기록: 성찬
ℹ️ URL 없음 → 텍스트 없이 기록: 원빈
ℹ️ URL 없음 → 텍스트 없이 기록: 승한
ℹ️ URL 없음 → 텍스트 없이 기록: 앤톤


### 그룹끼리 같은 논란/사건사고 페이지를 가지는 경우

시작: 해당 행 연예인 이름

끝: 같은 그룹의 다른 연예인 이름 가장 먼저 나오는 곳 이전까지

시작~끝 이전까지 슬라이싱

In [ ]:
import pandas as pd
import re

# 새 텍스트 자르기 함수
def trim_text_by_group_using_headers(text, current_name, group_names):
    if pd.isna(text):
        return text

    header_pattern_current = rf'\d+(?:\.\d+)*\.\s*{re.escape(current_name)}'
    current_match = re.search(header_pattern_current, text)

    if not current_match:
        return text

    start = current_match.start()

    other_names = [name for name in group_names if name != current_name]
    other_positions = []
    for other in other_names:
        pattern_other = rf'\d+(?:\.\d+)*\.\s*{re.escape(other)}'
        match_other = re.search(pattern_other, text[start+1:])
        if match_other:
            other_positions.append(start + 1 + match_other.start())

    end = min(other_positions) if other_positions else len(text)

    return text[start:end].strip()

# 크롤링된 데이터 불러오기
df = pd.read_csv("JYP논란_크롤링.csv")
artist_df = pd.read_csv("JYP_연예인목록.csv")

# 각 행마다 적용
for idx, row in df.iterrows():
    group = row['그룹']
    name = row['연예인 이름']
    full_text = row['텍스트']

    group_names = artist_df[artist_df['그룹'] == group]['연예인 이름'].tolist()
    trimmed = trim_text_by_group_using_headers(full_text, name, group_names)

    df.at[idx, '텍스트'] = trimmed

# 다시 저장
df.to_csv("JYP논란_크롤링.csv", index=False, encoding='utf-8-sig')
print("🎯 그룹 기준 목차-이름 매칭 텍스트 잘라내기 완료 및 저장")

🎯 그룹 기준 목차-이름 매칭 텍스트 잘라내기 완료 및 저장


In [ ]:
df = pd.read_csv("JYP논란_크롤링.csv")
artist_text = df[df['연예인 이름'] == '카리나']['텍스트'].values

print(artist_text[0])

3.1.
카리나
[편집]
3.1.1.
데뷔 전 악성 루머 피해
[편집]
aespa의 데뷔 직전에 온라인 커뮤니티 사이트를 통해 SM 연습생
유지민
(카리나의 본명)의 지인이라고 주장하는 누리꾼이 유지민과 나눈 문자 메시지 등을 공개해 논란이 일어났다. 해당 메시지에는 선배 가수 및 회사와 관련한 내용이 담겨 있었다.
메시지 조작 증거
이에 소속사 SM엔터테인먼트는 "유지민 양에 대한 악성 루머가 무분별하게 유포되고 있다"라며 해당 글이
허위 및 조작된 내용
임에도 불구하고, 끊임없이 확대 및 재생산되고 있다고 밝혔다. 또한 "온라인 상에서 벌어지는 인격 모독, 악성 루머 유포 행위 등 불법 행위에 대해 무관용 원칙하에 민·형사상 법적 조치를 취하여 대응할 것"이라고 밝히며 지난 10월 강남경찰서를 통해 유지민과 관련한
악의적 게시글을 작성하는 이들을 고소했다
고 전했다.
한국경제 기사
또한 카리나의 중고등학교 동창 등이 카리나를 둘러싼 악성 루머에 대해 부정하는 글을 올렸다.
#1
#2
#3
이 문단의 내용 중 전체 또는 일부는
카리나
문서의
uuid not found
판,
6번 문단
에서 가져왔습니다.
이전 역사 보러 가기


### 텍스트 정리

`소속사`, `연예인`, `사건 날짜`, `사건 내용`

`사건 내용`: 제목을 포함한 모든 텍스트

In [ ]:
import re
import pandas as pd

# 날짜 정규표현식 (예: 2020년 1월 2일 또는 2020년 01월 02일 등)
date_pattern = re.compile(r'\d{4}년\s?\d{1,2}월\s?\d{1,2}일')

# 결과 리스트 초기화
date_separated_data = []

# 각 연예인 텍스트에 대해 처리
for _, row in df.iterrows():
    full_text = str(row['텍스트'])  # ✅ 문자열로 변환 (NaN 방지)
    matches = list(date_pattern.finditer(full_text))

    if not matches:
        date_separated_data.append({
            '소속사': row['소속사'],
            '그룹': row['그룹'],
            '연예인 이름': row['연예인 이름'],
            '사건 날짜': '',
            '사건 내용': full_text.strip()
        })
    else:
        for i, match in enumerate(matches):
            start = match.start()
            end = matches[i + 1].start() if i + 1 < len(matches) else len(full_text)
            date = match.group()
            content = full_text[start:end].strip()
            date_separated_data.append({
                '소속사': row['소속사'],
                '그룹': row['그룹'],
                '연예인 이름': row['연예인 이름'],
                '사건 날짜': date,
                '사건 내용': content
            })

# 결과 DataFrame 생성 및 저장
date_df = pd.DataFrame(date_separated_data)
date_df.to_csv("JYP논란_크롤링.csv", index=False, encoding='utf-8-sig')

`사건 내용`: 마지막 문장이 끝나면(마지막 '~다.'이 나타나면) 그 다음 나머지 텍스트는 전부 삭제

In [ ]:
import pandas as pd
import re

# 파일 읽기
df = pd.read_csv('JYP논란_크롤링.csv')

# 텍스트 정제 함수
def keep_until_last_da(text):
    if pd.isna(text):
        return text

    # 줄바꿈 제거
    text = text.replace('\n', ' ').replace('\r', ' ').strip()

    # 마지막 '~다.' 위치 찾기
    match = list(re.finditer(r'다\.', text))
    if match:
        last_match = match[-1]
        cut_idx = last_match.end()
        return text[:cut_idx]
    else:
        # '~다.'가 없는 경우는 그대로 반환
        return text

# 적용
df['사건 내용'] = df['사건 내용'].apply(keep_until_last_da)

# 결과 저장
df.to_csv('JYP논란_크롤링.csv', index=False, encoding='utf-8-sig')

### 각 인물의 마지막 행에 주석 몰아서 들어간거 다 삭제

In [ ]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv('JYP논란_크롤링.csv')

# 각 인물의 마지막 사건 행만 복사
last_rows = df.groupby('연예인 이름').tail(1).copy()

# 인덱스 기준으로 전체 데이터 복사 (수정 반영용)
df_copy = df.copy()

# 각 인물 마지막 사건 텍스트에서 '[1]' 이후 삭제
for idx in last_rows.index:
    content = df_copy.at[idx, '사건 내용']
    if pd.notna(content):
        if '[1]' in content:
            df_copy.at[idx, '사건 내용'] = content.split('[1]')[0].strip()

# 저장
df_copy.to_csv('JYP논란_크롤링.csv', index=False, encoding='utf-8-sig')

### 불필요한 텍스트 다 삭제

(1) '[편집]' 이후 등장하는 문단 번호부터 삭제

(2) "자세한 내용은 ~ 문서를 참고하십시오" 문장 제거

(3) [숫자] 각주 제거

(4) '[편집]' 단어 자체 제거

(5) "이 문서의 내용 중 전체 또는 일부는" 이 포함된 문장 전체 제거

In [ ]:
import pandas as pd
import re

def remove_wiki_garbage(text):
    if pd.isna(text):
        return text

    # 줄바꿈 제거
    text = text.replace('\n', ' ').replace('\r', ' ')

    # 1. '[편집]' 앞 목차 패턴이 나타나면 → 거기서 자르기
    match = re.search(r'\d+\.\d+\..+?\[편집\]', text)
    if not match:
        match = re.search(r'\d+\..+?\[편집\]', text)
    if match:
        text = text[:match.start()]

    # 2. "자세한 내용은 ~ 문서를 참고하십시오" 제거
    text = re.sub(r'자세한 내용은 .*?문서를 참고하십시오\.?', '', text)

    # 3. [숫자] 각주 제거
    text = re.sub(r'\[\d+\]', '', text)

    # 4. '[편집]' 단어 제거
    text = text.replace('[편집]', '')

    # 5. "이 문서의 내용 중 전체 또는 일부는" 이 포함된 문장 전체 제거
    sentences = re.split(r'(?<=[.!?])\s+', text)
    filtered_sentences = [
        s for s in sentences if '이 문서의 내용 중 전체 또는 일부는' not in s
    ]
    text = ' '.join(filtered_sentences)

    # # 6. 'YYYY년 MM월 DD일' 형식 삭제
    # text = re.sub(r'[\s,\.]*\d{4}년\s*\d{1,2}월\s*\d{1,2}일[\s,\.]*', ' ', text)

    # 8. '파워링크 광고' 문구 포함 이후 텍스트 전체 제거 
    ad_idx = text.find('파워링크 광고')
    if ad_idx != -1:
        text = text[:ad_idx]

    # 7. 공백 정리
    return re.sub(r'\s+', ' ', text).strip()

# ▶ 2. CSV 파일 불러오기
input_path = 'JYP논란_크롤링.csv'
output_path = 'JYP논란_크롤링.csv'

df = pd.read_csv(input_path)

# ▶ 3. 정제 함수 적용
df['사건 내용'] = df['사건 내용'].apply(remove_wiki_garbage)

# ▶ 5. 결과 저장
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("정제가 완료되었습니다. 저장 파일:", output_path)


정제가 완료되었습니다. 저장 파일: SM논란_크롤링.csv


In [ ]:
import pandas as pd
import re

# 파일 불러오기
df = pd.read_csv('JYP논란_크롤링.csv')

# '~다.' 형태가 포함된 행만 남기기
pattern = r'\b[가-힣]*다\.'
df = df[df['사건 내용'].str.contains(pattern, regex=True, na=False)]

# 결과 저장
df.to_csv('JYP논란_크롤링.csv', index=False, encoding='utf-8-sig')

print("✅ '~다.' 형태 포함된 행만 남기고 저장 완료")

✅ '~다.' 형태 포함된 행만 남기고 저장 완료
